In [ ]:
import pandas as pd
import src.agent as agent

In [ ]:
df = pd.read_csv(filepath_or_buffer="data/data_faulty.csv")

In [ ]:
_df = df.copy()

columns_to_normalize = ['equipment_type', 'component']

_df, mappings = agent.normalize_columns_with_llm(_df, columns_to_normalize, model='qwen2.5')

In [ ]:
component = columns_to_normalize[1]
mappings[component]

In [ ]:
df[component].unique()

In [ ]:
_df[component].unique()